In [1]:
import pandas as pd
import os
import glob
import numpy as np

In [2]:
csv_list_1 = glob.glob('20240905_01/*.csv')
csv_list_2 = glob.glob('20240905_02/*.csv')
csv_list_3 = glob.glob('20240905_03/*.csv')

In [3]:
total_ = csv_list_1 + csv_list_2 + csv_list_3

In [4]:
df_bank_004    = pd.read_csv('20240905_02/마이데이터_데이터가공POC용_원천데이터추출(jira-735)_은행-004.csv')
df_elec_004_01 = pd.read_csv('20240905_02/마이데이터_데이터가공POC용_원천데이터추출(jira-735)_전금-004_202406분.csv')
df_elec_004_02 = pd.read_csv('20240905_02/마이데이터_데이터가공POC용_원천데이터추출(jira-735)_전금-004_202407분.csv')
df_elec_004_03 = pd.read_csv('20240905_02/마이데이터_데이터가공POC용_원천데이터추출(jira-735)_전금-004_202408분.csv')
df_elec_103    = pd.read_csv('20240905_03/마이데이터_데이터가공POC용_원천데이터추출(jira-735)_전금-103.csv')
df_prepaid_004 = pd.read_csv('20240905_02/마이데이터_데이터가공POC용_원천데이터추출(jira-735)_선불-004_카드.csv')
df_card_005    = pd.read_csv('20240905_03/마이데이터_데이터가공POC용_원천데이터추출(jira-735)_카드-005.csv')
df_card_008    = pd.read_csv('20240905_03/마이데이터_데이터가공POC용_원천데이터추출(jira-735)_카드-008.csv')
# '고객식별자', '금액', '결제시간' 동일하고 취소결제의 '정정 또는 승인취소 일시'가 늦은 경우 결제내역에서 제거
df_tele_004    = pd.read_csv('20240905_03/마이데이터_데이터가공POC용_원천데이터추출(jira-735)_통신_004.csv')

In [5]:
# combine data
df_elec_004    = pd.concat([df_elec_004_01, df_elec_004_02, df_elec_004_03], ignore_index = True)

### 자전거래 잡기 (CASE 1: 카드-체크카드, 전금-결제수단식별코드: 계좌/선불/무통장/타기관간편결제/기타)

### 전금데이터 전처리

In [87]:
# 전금-004 데이터 카피
elec_004_copy = df_elec_004
# 전금-103 데이터 카피
elec_103_copy = df_elec_103

In [ ]:
# 전금-004 데이터 처리
# filtered_elec_004 = df_elec_004

# filter bank related transaction
# filtered_elec_004               = df_elec_004[df_elec_004['ETFN_TX_TYCD']] # 카드충전/
# filtered_elec_004['TX_YMD']     = filtered_elec_004['TX_DTTM'].astype(str).str[:8].astype('int64')
# filtered_elec_004['TX_YMDH']    = filtered_elec_004['TX_DTTM'].astype(str).str[:10].astype('int64')

In [88]:
print(f"elec_004: {elec_004_copy.shape}", f"elec_103: {elec_103_copy.shape}")

elec_004: (2598718, 25) elec_103: (247687, 27)


In [89]:
# preprocessing
elec_004_copy = elec_004_copy[~elec_004_copy['ETFN_TX_TYCD'].isin([5103,5113,5301,5302])] # 포인트결제/적립관련 거래유형 제거

# 5201 계좌송금   - 5109 기타충전
# 5901 기타(입금) - 5902 기타(출금)   -- 금액이 다른 경우도 존재 TX_PTCL_SRNO 참고
# 5501 결제      - 5502 결제취소     -- 결제내역취소로 간주 (TX_PTCL_SRNO 참고)
# 5502 결제취소   - 5902 기타(출금)   -- 결제취소와 동일한 가격으로 찾아서 취소해줘야댐
# 5501 결제      - 5901 기타(입금)  
# 5201 계좌송금   - 5202 계정송금
# 5202 계정송금   - 5102 계좌이체(충전) 
# 5502 결제취소   - 5501 결제
# 5201 계좌송금   - 5901 기타(입금)

# case 1: 취소거래 확인
dup_cols_1 = ['MBR_NO', 'TX_DTTM', 'FNIT_CODE', 'ENCD_TX_CTP_DTVL','FRCS_NM','FRCS_BISNO']
dup_cols_2 = ['MBR_NO', 'TX_DTTM', 'FNIT_CODE', 'ENCD_TX_CTP_DTVL','FRCS_NM','FRCS_BISNO','ETFN_TX_TYCD']

duplicated_004   = elec_004_copy[(elec_004_copy.duplicated(subset=dup_cols_1, keep=False))]
duplicated_004_2 = duplicated_004[(duplicated_004.duplicated(subset=dup_cols_2, keep=False))]
duplicated_004_2 = duplicated_004_2[['MBR_NO', 'TX_DTTM', 'FNIT_CODE', 'ENCD_TX_CTP_DTVL','FRCS_NM','FRCS_BISNO','ETFN_TX_TYCD']]

duplicated_004_3 = pd.merge(duplicated_004, duplicated_004_2, on = ['MBR_NO', 'TX_DTTM', 'FNIT_CODE', 'ENCD_TX_CTP_DTVL','FRCS_NM','FRCS_BISNO','ETFN_TX_TYCD'], how = 'outer', indicator=True)
duplicated_004_3 = duplicated_004_3[duplicated_004_3['_merge'] == 'left_only'].drop('_merge', axis=1).reset_index(drop=True)

In [90]:
# 전금-103 데이터 처리
elec_103_copy = elec_103_copy[elec_103_copy['STMT_METH_DTGS_CODE'] != 3] # 선불내역 제거

elec_103_copy = elec_103_copy[['MBR_NO', 'STMT_DTTM', 'ETFN_TX_TYCD', 'STMT_NO', 'CRC_CODE', 'ETFN_STMT_AMT',
       'STMT_METH_ORG_CODE', 'STMT_METH_DTGS_CODE', 'ENCD_STMT_METH_DTVL', 'CARD_PD_NM', 
       'ITMT_MMCNT', 'TX_MEMO', 'FRCS_NM', 'FRCS_BISNO', 'PD_TITLE_CTS', 'PD_CLCD', 'ETFN_STMT_MTCD']]

# convert time
elec_103_copy['STMT_DTTM_time']  = np.where(len(elec_103_copy['STMT_DTTM']) == 8, pd.to_datetime(elec_103_copy['STMT_DTTM'], format='%Y%m%d', errors='coerce'), pd.to_datetime(elec_103_copy['STMT_DTTM'], format='%Y%m%d%H%M%S', errors='coerce'))

# STEP 1: 결제취소내역 삭제 (시간동일)
dup_cols_1 = ['MBR_NO', 'STMT_DTTM_time', 'FRCS_NM', 'FRCS_BISNO','ETFN_STMT_AMT']
dup_cols_2 = ['MBR_NO', 'STMT_DTTM_time', 'FRCS_NM', 'FRCS_BISNO','ETFN_STMT_AMT','ETFN_TX_TYCD']

# dup_cols_1 컬럼리스트 동일
duplicated_103   = elec_103_copy[(elec_103_copy.duplicated(subset=dup_cols_1, keep=False))]
# dup_cols_2 컬럼리스트 동일 (결제승인내역이 같은 시간에 이루어진 경우)
duplicated_103_1 = duplicated_103[(duplicated_103.duplicated(subset=dup_cols_2, keep=False))]

# duplicated 제거를 위한 컬럼 선택
duplicated_103_1 = duplicated_103_1[['MBR_NO', 'STMT_DTTM_time', 'FRCS_NM', 'FRCS_BISNO','ETFN_STMT_AMT','ETFN_TX_TYCD']]


duplicated_103_2 = pd.merge(duplicated_103, duplicated_103_1, on = ['MBR_NO', 'STMT_DTTM_time', 'FRCS_NM', 'FRCS_BISNO','ETFN_STMT_AMT','ETFN_TX_TYCD'], how = 'outer', indicator=True)
duplicated_103_3 = duplicated_103_2[duplicated_103_2['_merge'] == 'left_only'].drop('_merge', axis=1).reset_index(drop=True)

# 잘못된 데이터 제거하기
duplicated_103_4 = duplicated_103_3.groupby(['MBR_NO', 'STMT_DTTM_time', 'ETFN_STMT_AMT']).size().reset_index(name='count')
duplicated_103_4 = duplicated_103_4[duplicated_103_4['count'] % 2 != 0]
duplicated_103_4 = duplicated_103_4[['MBR_NO', 'STMT_DTTM_time', 'ETFN_STMT_AMT']]

duplicated_103_one = pd.merge(duplicated_103_3, duplicated_103_4, on = ['MBR_NO', 'STMT_DTTM_time', 'ETFN_STMT_AMT'], how = 'outer', indicator=True)
duplicated_103_one = duplicated_103_one[duplicated_103_one['_merge'] == 'left_only'].drop('_merge', axis=1).reset_index(drop=True)

# 결제코드 조합 확인
answer = ['1,3', '1,2']
duplicated_103_one['ETFN_TX_TYCD_concat'] = duplicated_103_one.groupby(['MBR_NO', 'STMT_DTTM_time', 'ETFN_STMT_AMT'])['ETFN_TX_TYCD'].transform(lambda x: ','.join(sorted(x.astype(str))))
duplicated_103_one = duplicated_103_one[duplicated_103_one['ETFN_TX_TYCD_concat'].isin(answer)]


# select columns
duplicated_103_one = duplicated_103_one[duplicated_103_one['ETFN_TX_TYCD']==1]
duplicated_103_one = duplicated_103_one[['MBR_NO','STMT_DTTM', 'STMT_DTTM_time', 'STMT_NO', 'ETFN_STMT_AMT', 'TX_MEMO', 'FRCS_NM', 'FRCS_BISNO']]
duplicated_103_one = duplicated_103_one.drop_duplicates()

In [91]:
# STEP 2: 시간이 다른 경우

candidates = elec_103_copy

# 기존 잡은 데이터 제거
candidates = pd.merge(candidates, duplicated_103_one, on = ['MBR_NO','STMT_DTTM','STMT_DTTM_time', 'STMT_NO', 'ETFN_STMT_AMT', 'TX_MEMO', 'FRCS_NM', 'FRCS_BISNO'], how = 'outer', indicator=True)
candidates = candidates[candidates['_merge'] == 'left_only'].drop('_merge', axis=1).reset_index(drop=True)

# split data
cancelled_candidates = candidates[candidates['ETFN_TX_TYCD'].isin([2,3])].reset_index(drop=True)
approved_candidates  = candidates[candidates['ETFN_TX_TYCD'].isin([1])].reset_index(drop=True)

# 해당 데이터와 매핑되는 결제내역 결과 건수 확인
cancelled_summary    = cancelled_candidates[['MBR_NO', 'STMT_DTTM', 'STMT_DTTM_time', 'STMT_NO', 'ETFN_STMT_AMT', 'STMT_METH_DTGS_CODE', 
                                             'TX_MEMO', 'FRCS_NM', 'FRCS_BISNO', 'PD_TITLE_CTS', 'PD_CLCD']]

approved_summary    = approved_candidates[['MBR_NO', 'STMT_DTTM', 'STMT_DTTM_time', 'STMT_NO', 'ETFN_STMT_AMT', 'STMT_METH_DTGS_CODE', 
                                             'TX_MEMO', 'FRCS_NM', 'FRCS_BISNO', 'PD_TITLE_CTS', 'PD_CLCD']]

merged_summary      =  pd.merge(cancelled_summary, approved_summary, on=['MBR_NO', 'ETFN_STMT_AMT', 'STMT_METH_DTGS_CODE', 'TX_MEMO', 'FRCS_NM', 'FRCS_BISNO', 'PD_TITLE_CTS', 'PD_CLCD'], 
                                how='left', indicator=True)

# select inner join only
result_inner         = merged_summary[merged_summary['_merge']=='both']
result_inner.columns = ['MBR_NO', 'STMT_DTTM_cancel', 'STMT_DTTM_cancel_time', 'STMT_NO_cancel', 'ETFN_STMT_AMT', 'STMT_METH_DTGS_CODE',
       'TX_MEMO', 'FRCS_NM', 'FRCS_BISNO', 'PD_TITLE_CTS', 'PD_CLCD',
       'STMT_DTTM_approved', 'STMT_DTTM_approved_time', 'STMT_NO_approved', '_merge']

result_inner['compare'] = np.where(result_inner['STMT_DTTM_approved_time'] < result_inner['STMT_DTTM_cancel_time'], 'right', 'wrong')

# 취소시간 < 결제시간 내역 삭제
result_inner = result_inner[result_inner['compare'] == 'right']

# get the right pair if it is more than two
result_inner['time_diff'] = round((result_inner['STMT_DTTM_cancel_time'] - result_inner['STMT_DTTM_approved_time']).dt.total_seconds())

# minimum time difference
result_inner['min_diff'] = result_inner.groupby(['MBR_NO','STMT_DTTM_cancel','ETFN_STMT_AMT','FRCS_NM','FRCS_BISNO','PD_TITLE_CTS'])['time_diff'].transform('min')

# get the value with the minimum time difference
duplicated_103_two  = result_inner[result_inner['min_diff'] == result_inner['time_diff']]

duplicated_103_two = result_inner[['MBR_NO','STMT_DTTM_approved', 'STMT_DTTM_approved_time', 'STMT_NO_approved', 'ETFN_STMT_AMT', 'TX_MEMO', 'FRCS_NM', 'FRCS_BISNO']]
duplicated_103_two.columns = ['MBR_NO','STMT_DTTM', 'STMT_DTTM_time', 'STMT_NO', 'ETFN_STMT_AMT', 'TX_MEMO', 'FRCS_NM', 'FRCS_BISNO']

/var/folders/fv/yzzct79s5_n1jybq64nt5ls00000gn/T/ipykernel_1626/2561027949.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_inner['compare'] = np.where(result_inner['STMT_DTTM_approved_time'] < result_inner['STMT_DTTM_cancel_time'], 'right', 'wrong')


In [92]:
# combine two dataframes
cancelled_elec_103_df = pd.concat([duplicated_103_one, duplicated_103_two], ignore_index=True)

In [93]:
# cancel out cancelled transactions
approved_elec_103_df = elec_103_copy[elec_103_copy['ETFN_TX_TYCD'].isin([1])].reset_index(drop=True)

approved_elec_103_df = pd.merge(approved_elec_103_df, cancelled_elec_103_df, on = ['MBR_NO','STMT_DTTM', 'STMT_DTTM_time', 'STMT_NO', 'ETFN_STMT_AMT', 'TX_MEMO', 'FRCS_NM', 'FRCS_BISNO'], how = 'outer', indicator=True)
approved_elec_103_df = approved_elec_103_df[approved_elec_103_df['_merge'] == 'left_only'].drop('_merge', axis=1).reset_index(drop=True)

In [13]:
# 카드데이터만 추출
approved_elec_103_df = approved_elec_103_df[approved_elec_103_df['STMT_METH_DTGS_CODE']==1]

# 필요컬럼 추출
# approved_elec_103_df = approved_elec_103_df[['MBR_NO', 'STMT_DTTM', 'ETFN_STMT_AMT', 'FRCS_NM', 'FRCS_BISNO', 'PD_TITLE_CTS', 'PD_CLCD', 'STMT_DTTM_time']]

### 카드

In [14]:
card_008_copy = df_card_008

# 신용카드 내역 추출
card_008_copy = card_008_copy[card_008_copy['CARD_KNCD']==1]

card_008_copy.shape

(315739, 18)

In [15]:
# convert time
card_008_copy['APV_DTTM_time']  = np.where(len(card_008_copy['APV_DTTM']) == 8, pd.to_datetime(card_008_copy['APV_DTTM'], format='%Y%m%d', errors='coerce'), pd.to_datetime(card_008_copy['APV_DTTM'], format='%Y%m%d%H%M%S', errors='coerce'))

# STEP 1: 결제취소내역 삭제
# STEP 1-1: 승인시간 및 승인번호 기준 데이터 필터링
dup_cols_1 = ['MBR_NO', 'CARD_SRNO', 'CARD_APV_NO', 'FRCS_BISNO', 'FRCS_NM', 'CARD_KNCD']
dup_cols_2 = ['MBR_NO', 'CARD_SRNO', 'CARD_APV_NO', 'FRCS_BISNO', 'FRCS_NM', 'CARD_KNCD','CARD_STMT_STCD']
# dup_cols_3 = ['MBR_NO', 'CARD_SRNO', 'FRCS_BISNO', 'FRCS_NM', 'CARD_KNCD','CARD_STMT_STCD']

duplicated_008 = card_008_copy[(card_008_copy.duplicated(subset=dup_cols_1, keep=False))]
duplicated_008_1 = duplicated_008[(duplicated_008.duplicated(subset=dup_cols_2, keep=False))]
duplicated_008_1 = duplicated_008_1[['MBR_NO', 'CARD_SRNO', 'CARD_APV_NO', 'FRCS_BISNO', 'FRCS_NM', 'CARD_KNCD','CARD_STMT_STCD']]

duplicated_008_2 = pd.merge(duplicated_008, duplicated_008_1, on = ['MBR_NO', 'CARD_SRNO', 'CARD_APV_NO', 'FRCS_BISNO', 'FRCS_NM', 'CARD_KNCD','CARD_STMT_STCD'], how = 'outer', indicator=True)
duplicated_008_3 = duplicated_008_2[duplicated_008_2['_merge'] == 'left_only'].drop('_merge', axis=1).reset_index(drop=True)

# STEP 1-2: 잘못된 데이터 제거

# 잘못된 데이터 제거하기
duplicated_008_4 = duplicated_008_3.groupby(['MBR_NO', 'CARD_SRNO', 'CARD_APV_NO']).size().reset_index(name='count')
duplicated_008_4 = duplicated_008_4[duplicated_008_4['count'] % 2 != 0]
duplicated_008_4 = duplicated_008_4[['MBR_NO', 'CARD_SRNO', 'CARD_APV_NO']]

duplicated_008_one = pd.merge(duplicated_008_3, duplicated_008_4, on = ['MBR_NO', 'CARD_SRNO', 'CARD_APV_NO'], how = 'outer', indicator=True)
duplicated_008_one = duplicated_008_one[duplicated_008_one['_merge'] == 'left_only'].drop('_merge', axis=1).reset_index(drop=True)

# 결제코드 조합 확인
answer = ['1,2']
duplicated_008_one['CARD_STMT_STCD_concat'] = duplicated_008_one.groupby(['MBR_NO', 'CARD_SRNO', 'CARD_APV_NO'])['CARD_STMT_STCD'].transform(lambda x: ','.join(sorted(x.astype(str))))
duplicated_008_one = duplicated_008_one[duplicated_008_one['CARD_STMT_STCD_concat'].isin(answer)]

# select columns
duplicated_008_one = duplicated_008_one[duplicated_008_one['CARD_STMT_STCD']==1]
duplicated_008_one = duplicated_008_one[['MBR_NO', 'CARD_SRNO', 'CARD_APV_NO']]
duplicated_008_one = duplicated_008_one.drop_duplicates()

cancelled_card_008_df = duplicated_008_one

/var/folders/fv/yzzct79s5_n1jybq64nt5ls00000gn/T/ipykernel_1626/669769403.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  card_008_copy['APV_DTTM_time']  = np.where(len(card_008_copy['APV_DTTM']) == 8, pd.to_datetime(card_008_copy['APV_DTTM'], format='%Y%m%d', errors='coerce'), pd.to_datetime(card_008_copy['APV_DTTM'], format='%Y%m%d%H%M%S', errors='coerce'))


In [16]:
# cancel out cancelled transactions
approved_card_008_df = card_008_copy[card_008_copy['CARD_STMT_STCD'].isin([1])].reset_index(drop=True)

approved_card_008_df = pd.merge(approved_card_008_df, cancelled_card_008_df, on = ['MBR_NO', 'CARD_SRNO', 'CARD_APV_NO'], how = 'outer', indicator=True)
approved_card_008_df = approved_card_008_df[approved_card_008_df['_merge'] == 'left_only'].drop('_merge', axis=1).reset_index(drop=True)

In [17]:
approved_card_008_df = approved_card_008_df[['MBR_NO', 'CARD_SRNO', 'CARD_APV_SRNO', 'APV_DTTM', 'CARD_APV_NO',
       'CARD_STMT_STCD', 'CARD_KNCD', 'APV_UPD_CCL_DTTM', 'FRCS_NM', 'USE_AMT',
       'UPD_USE_AMT', 'WHL_ITMT_RD_NO', 'FRCS_BISNO', 'APV_DTTM_time']]

In [197]:
# approved_card_008_df['APV_DTTM_after_2min'] = approved_card_008_df['APV_DTTM_time'] + pd.Timedelta(minutes=2)
# approved_card_008_df['APV_DTTM_after_5min'] = approved_card_008_df['APV_DTTM_time'] + pd.Timedelta(minutes=5)
# approved_card_008_df['APV_DTTM_after_10min'] = approved_card_008_df['APV_DTTM_time'] + pd.Timedelta(minutes=10)
# approved_card_008_df['APV_DTTM_after_1days'] = approved_card_008_df['APV_DTTM_time'] + pd.Timedelta(days=1)
# approved_card_008_df['APV_DTTM_after_2days'] = approved_card_008_df['APV_DTTM_time'] + pd.Timedelta(days=2)
# approved_card_008_df['APV_DTTM_after_3days'] = approved_card_008_df['APV_DTTM_time'] + pd.Timedelta(days=3)
# approved_card_008_df['APV_DTTM_after_7days'] = approved_card_008_df['APV_DTTM_time'] + pd.Timedelta(days=7)

In [18]:
# CASE 1: 결제시간과 결제금액까지 정확히 동일한 경우
result_df_1 = pd.merge(approved_elec_103_df,approved_card_008_df, left_on = ['MBR_NO', 'ETFN_STMT_AMT','STMT_DTTM_time'], right_on = ['MBR_NO','USE_AMT','APV_DTTM_time'], how='inner', indicator=True)
# result_df_1.to_excel('exact_match.xlsx', index=False)

In [19]:
copy_result_1         = result_df_1[['MBR_NO', 'STMT_DTTM', 'ETFN_STMT_AMT', 'FRCS_NM_x', 'FRCS_BISNO_x','PD_TITLE_CTS']]
copy_result_1.columns = ['MBR_NO', 'STMT_DTTM', 'ETFN_STMT_AMT', 'FRCS_NM', 'FRCS_BISNO','PD_TITLE_CTS']

# CASE 2: 결제금액만 정확히 동일한 경우
non_exact_df = pd.merge(approved_elec_103_df, copy_result_1, on = ['MBR_NO', 'STMT_DTTM', 'ETFN_STMT_AMT', 'FRCS_NM', 'FRCS_BISNO','PD_TITLE_CTS'], how = 'outer', indicator=True)
non_exact_df = non_exact_df[non_exact_df['_merge'] == 'left_only'].drop('_merge', axis=1).reset_index(drop=True)

result_df_2 = pd.merge(non_exact_df,approved_card_008_df, left_on = ['MBR_NO','ETFN_STMT_AMT'], right_on = ['MBR_NO','USE_AMT'], how='inner', indicator=True)

# check whether APV_DTTM_time is faster than STMT_DTTM_time
result_df_2['time_check'] = np.where(result_df_2['APV_DTTM_time'] < result_df_2['STMT_DTTM_time'], 'right', 'wrong')

result_df_2 = result_df_2[result_df_2['time_check']=='right']

# get the right pair if it is more than two
result_df_2['time_diff'] = abs(round((result_df_2['STMT_DTTM_time'] - result_df_2['APV_DTTM_time']).dt.total_seconds()))

# minimum time difference
result_df_2['min_diff'] = result_df_2.groupby(['MBR_NO','STMT_DTTM','ETFN_STMT_AMT','FRCS_NM_x','FRCS_BISNO_x','PD_TITLE_CTS'])['time_diff'].transform('min')

# get the value with the minimum time difference
result_df_2  = result_df_2[result_df_2['min_diff'] == result_df_2['time_diff']].reset_index(drop=True)

In [20]:
result_df_2 = result_df_2.drop_duplicates().reset_index(drop=True)

In [224]:
# result_df_2.to_excel('time_diff.xlsx', index=False)

### 은행

In [22]:
# copy the bank df
bank_004_copy = df_bank_004

In [24]:
bank_004_copy = bank_004_copy[['TX_PTCL_SRNO', 'MBR_NO', 'BANK_ACT_SRNO', 'TX_DTTM', 'TX_NO',
       'CRC_CODE', 'ACT_TX_TYCD', 'BANK_TX_DIV_NM', 'TX_AMT', 'AFTX_BLC','TX_RD_NO', 'SUMRY']]

Index(['TX_PTCL_SRNO', 'MBR_NO', 'BANK_ACT_SRNO', 'TX_DTTM', 'TX_NO',
       'CRC_CODE', 'ACT_TX_TYCD', 'BANK_TX_DIV_NM', 'TX_AMT', 'AFTX_BLC',
       'TX_RD_NO', 'SUMRY', 'GTHR_EXEC_DTTM', 'RGR_ID', 'RG_DTTM', 'CGR_ID',
       'CG_DTTM'],
      dtype='object')

In [371]:
"""
ACT_TX_TYCD
01 : 신규
02 : 출금 - 04: 정정(입금) / 06: 출금취소(입금)
03 : 입금 - 05: 정정(출금) / 07: 입금취소(출금)
98 : 기타(입금)
99 : 기타(출금)
"""

bank_004_copy['TX_DTTM_time']  = np.where(len(bank_004_copy['TX_DTTM']) == 8, pd.to_datetime(bank_004_copy['TX_DTTM'], format='%Y%m%d', errors='coerce'), pd.to_datetime(bank_004_copy['TX_DTTM'], format='%Y%m%d%H%M%S', errors='coerce'))

# EDA
bank_004_copy_1 = bank_004_copy[bank_004_copy['ACT_TX_TYCD'].isin([6])] # 출금-정정(출금) / 출금-출금취소(입금)
bank_004_copy_2 = bank_004_copy[bank_004_copy['ACT_TX_TYCD'].isin([2])] # 출금-정정(출금) / 출금-출금취소(입금)
bank_004_copy_1.shape

/var/folders/fv/yzzct79s5_n1jybq64nt5ls00000gn/T/ipykernel_1626/3489763578.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bank_004_copy['TX_DTTM_time']  = np.where(len(bank_004_copy['TX_DTTM']) == 8, pd.to_datetime(bank_004_copy['TX_DTTM'], format='%Y%m%d', errors='coerce'), pd.to_datetime(bank_004_copy['TX_DTTM'], format='%Y%m%d%H%M%S', errors='coerce'))


(2221, 13)

In [383]:
bank_004_copy[(bank_004_copy['MBR_NO']==237696) & (bank_004_copy['BANK_ACT_SRNO']==1217812) & (bank_004_copy['TX_AMT']==70000) ]

,TX_PTCL_SRNO,MBR_NO,BANK_ACT_SRNO,TX_DTTM,TX_NO,CRC_CODE,ACT_TX_TYCD,BANK_TX_DIV_NM,TX_AMT,AFTX_BLC,TX_RD_NO,SUMRY,TX_DTTM_time
153151,247610490,237696,1217812,20240605194203,2N1,KRW,2,NH콕송금,70000.0,0.0,0,임현조,2024-06-05 19:42:03
153154,247610493,237696,1217812,20240604200656,2N1,KRW,2,NH콕송금,70000.0,9240.0,0,이현국_세피나,2024-06-04 20:06:56


In [378]:
# ACT_TX_TYCD == 4 & BANK_TX_DIV_NM == '지급정정' then TX_AMT is negative, SUMRY는 동일
# SUMRY와 ACT_TX_TYCD, TX_AMT 활용하면 될듯?

bank_004_copy[bank_004_copy['ACT_TX_TYCD'].isin([4])].reset_index(drop=True)

,TX_PTCL_SRNO,MBR_NO,BANK_ACT_SRNO,TX_DTTM,TX_NO,CRC_CODE,ACT_TX_TYCD,BANK_TX_DIV_NM,TX_AMT,AFTX_BLC,TX_RD_NO,SUMRY,TX_DTTM_time
0,274583396,220553,916830,20240807170352,7544,KRW,4,기타,3680.0,2949085.0,0,,2024-08-07 17:03:52
1,274583397,220553,916830,20240807170340,7543,KRW,4,기타,21230.0,2945405.0,0,,2024-08-07 17:03:40
2,247610492,237696,1217812,20240604200657,3N1,KRW,4,지급정정,-70000.0,79240.0,0,이현국_세피나,2024-06-04 20:06:57
3,232019076,259668,1098004,20240703154824,20240703000000000401,KRW,4,NH카드,30000.0,52432.0,0,다온에너비스(주)해양주유소,2024-07-03 15:48:24
4,221522554,270629,1153965,20240610211415,20240610000000000301,KRW,4,NH카드,5400.0,428168.0,0,G마켓_SmilePay,2024-06-10 21:14:15
5,230334834,270629,1153965,20240623203245,20240623000000000901,KRW,4,NH카드,7890.0,133301.0,0,쿠팡(와우멤버십),2024-06-23 20:32:45
6,239837716,270629,1153965,20240711210545,20240711000000000601,KRW,4,NH카드,6050.0,577257.0,0,11번가,2024-07-11 21:05:45
7,246329622,270629,1153965,20240720203011,20240720000000000801,KRW,4,대외거래,8200.0,506604.0,0,다이소,2024-07-20 20:30:11
8,266302451,270629,1153965,20240818174818,20240818000000000401,KRW,4,NH카드,20000.0,662023.0,0,쿠팡이츠,2024-08-18 17:48:18
9,269962569,270629,1153965,20240826134949,20240826000000000201,KRW,4,NH카드,3200.0,719055.0,0,선물하기_카카오페이,2024-08-26 13:49:49


In [201]:
# 출금-출금취소(입금) : MBR_NO, BANK_ACT_SRNO, BANK_TX_DIV_NM, TX_AMT, SUMRY
bank_004_copy_1 = bank_004_copy_1[['MBR_NO','TX_DTTM', 'TX_DTTM_time', 'BANK_ACT_SRNO', 'TX_AMT', 'SUMRY']]

bank_004_copy_1['TX_DTTM_min-1'] = bank_004_copy_1['TX_DTTM_time'] - pd.Timedelta(minutes=1)
bank_004_copy_1['TX_DTTM_min-5'] = bank_004_copy_1['TX_DTTM_time'] - pd.Timedelta(minutes=5)
bank_004_copy_1['TX_DTTM_-1']    = bank_004_copy_1['TX_DTTM_time'] - pd.Timedelta(days=1)
# bank_004_copy_1['TX_DTTM_-2']    = bank_004_copy_1['TX_DTTM_time'] - pd.Timedelta(days=2)

In [241]:
from tqdm import tqdm

match_result   = []
poten_result_1 = []
poten_result_2 = []
poten_result_3 = []

approved_df = bank_004_copy_2

for idx_1, row_1 in tqdm(bank_004_copy_1.iterrows(), total=len(bank_004_copy_1)):
    df = approved_df[approved_df['MBR_NO'] == row_1['MBR_NO']]
    for idx_2, row_2 in df.iterrows():
        if ((row_1['BANK_ACT_SRNO'] == row_1['BANK_ACT_SRNO']) & (row_1['TX_AMT'] == row_2['TX_AMT']) & (row_1['SUMRY'] == row_2['SUMRY'])):
            # time difference check
            if row_1['TX_DTTM_time'] == row_2['TX_DTTM_time']:
                match_result.append((idx_1,idx_2))
                used_rows.append(idx_2)
                # print(f"match_result : {len(match_result)}")
                approved_df = approved_df.drop(idx_2, errors='ignore')
                df = df.drop(idx_2, errors='ignore')
                break
            elif ((row_2['TX_DTTM_time'] < row_1['TX_DTTM_time']) & (row_2['TX_DTTM_time'] > row_1['TX_DTTM_-1'])):
                poten_result_1.append((idx_1,idx_2))
                used_rows.append(idx_2)
                # print(f"poten_result : {len(poten_result_1)}")
                approved_df = approved_df.drop(idx_2, errors='ignore')
                df = df.drop(idx_2, errors='ignore')
                break

        elif ((row_1['BANK_ACT_SRNO'] == row_1['BANK_ACT_SRNO']) & 
             (row_1['TX_AMT'] == row_2['TX_AMT']) & 
             (row_2['TX_DTTM_time'] < row_1['TX_DTTM_time']) & 
             (row_2['TX_DTTM_time'] > row_1['TX_DTTM_-1'])):
            if (row_2['TX_DTTM_time'] < row_1['TX_DTTM_time']) & (row_2['TX_DTTM_time'] > row_1['TX_DTTM_min-1'])):
                poten_result_1.append((idx_1,idx_2))
                approved_df = approved_df.drop(idx_2, errors='ignore')
                df = df.drop(idx_2, errors='ignore')
                break
            elif (row_2['TX_DTTM_time'] < row_1['TX_DTTM_time']) & (row_2['TX_DTTM_time'] > row_1['TX_DTTM_min-5'])):
                poten_result_2.append((idx_1,idx_2))
                approved_df = approved_df.drop(idx_2, errors='ignore')
                df = df.drop(idx_2, errors='ignore')
                break
            else:
                poten_result_3.append((idx_1,idx_2))
                approved_df = approved_df.drop(idx_2, errors='ignore')
                df = df.drop(idx_2, errors='ignore')
                break
        else:
            approved_df = approved_df.drop(idx_2, errors='ignore')
    
    # print(f"{row_1['MBR_NO']} is done")

100%|███████████████████████████████████████| 2221/2221 [29:18<00:00,  1.26it/s]


In [267]:
print(f"match_result: {len(match_result)}")
print(f"poten_result_1: {len(poten_result_1)}")
print(f"poten_result_2: {len(poten_result_2)}")
print(f"poten_result_3: {len(poten_result_3)}")

match_result: 30
poten_result_1: 1250
poten_result_2: 56
poten_result_3: 6


In [370]:
i = 38
print(bank_004_copy_1.iloc[poten_result_2[i][0],])
print("----------------------------------------")
print(bank_004_copy_2.iloc[poten_result_2[i][1],])

MBR_NO                        329223
TX_DTTM               20240601111103
TX_DTTM_time     2024-06-01 11:11:03
BANK_ACT_SRNO                1533580
TX_AMT                      150000.0
SUMRY                        ATM출금취소
TX_DTTM_-1       2024-05-31 11:11:03
TX_DTTM_-2       2024-05-30 11:11:03
Name: 1740, dtype: object
----------------------------------------
TX_PTCL_SRNO                244027182
MBR_NO                         329223
BANK_ACT_SRNO                 1533580
TX_DTTM                20240601111054
TX_NO                             1d1
CRC_CODE                          KRW
ACT_TX_TYCD                         2
BANK_TX_DIV_NM                   카드출금
TX_AMT                       150000.0
AFTX_BLC                     406092.0
TX_RD_NO                            0
SUMRY                           ATM출금
TX_DTTM_time      2024-06-01 11:10:54
Name: 316472, dtype: object
